# Load data
_Note_: tempo = bpm

In [2]:
import pandas as pd
sample = pd.read_csv("datasets/full_data.csv", nrows=3)
sample.columns

Index(['Unnamed: 0', 'track', 'artist', 'seeds', 'spotify_id', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature'],
      dtype='object')

In [3]:
KEEP_COLS_MASTER = ["track", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["track", "artist", "tempo", "key", "loudness", "time_signature", "duration_ms"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("datasets/full_data.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index("track", inplace=True)
songs_df

/tmp/ipykernel_161884/842477347.py:9: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  songs_df = pd.read_csv("datasets/full_data.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)


,artist,key,loudness,tempo,duration_ms,time_signature
track,,,,,,
'Till I Collapse,Eminem,1,-3.237,171.447,297787,4
St. Anger,Metallica,2,-2.642,185.252,441133,4
Speedin',Rick Ross,1,-4.226,100.059,204960,4
Bamboo Banga,M.I.A.,9,-4.554,125.984,298360,4
Die MF Die,Dope,5,-3.524,126.020,186067,4
...,...,...,...,...,...,...
Secret,Quietdrive,0,-5.126,167.996,258373,4
The Last of the Rest Was the End,Medications,2,-5.774,144.844,324000,3
Lovechild,Daniel Lanois,10,-20.091,79.476,516280,4


Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [4]:
# print(songs_df[songs_df["release_date"]=="1993"])
# # credit https://stackoverflow.com/a/54312941
# songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
# songs_df["release_date"]

Load the V, A, D dimension values from the original MuSe dataset and add them to the rest of the data.
^ Join V, A, D columns by key columns title and author.

In [5]:
import pandas as pd

orig_dataset = pd.read_csv("datasets/muse_v3.csv")
orig_dataset.columns
orig_dataset.set_index("track", inplace=True)

In [6]:
VAL_COLS = ["valence_tags","arousal_tags","dominance_tags"]
songs_df = songs_df.join(orig_dataset[VAL_COLS], how="inner")

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [7]:
# How much should we take for training, testing?
songs_df.index.size

0.7 * songs_df.index.size

77934.5

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

DROP_COLS = VAL_COLS+["artist"]
# use a constant random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=DROP_COLS), songs_df[VAL_COLS],
                                                    test_size=0.2, random_state=42)

## Set up Weights & Biases for Training
Easily visualize training, track performance over iterations and try out a bunch of different values of hyperparameters.

In [10]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: t0m_x. Use `wandb login --relogin` to force relogin


True

# Model
## Multi-Layer Perceptron
[PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

[Building a Regression Model in PyTorch](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)

[ChatGPT Explaining The PyTorch Tutorial...](https://chat.openai.com/share/424f5717-901e-4dac-9f71-f30b1e4ab6de)

In [ ]:
# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(solver='lbfgs', hidden_layer_sizes=())


# mlp_regression = mlp.fit(X_train, y_train)
# print(mlp_regression.score(X_test, y_test))

In [11]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

ModuleNotFoundError: No module named 'torch'

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 1),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.ReLU(),
            nn.Linear(3, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

## Histogram-Based Gradient Boosted Tree
- [scikit-learn tutorial](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting)
- [regressor API reference](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor)

In [23]:
wandb.init(
    project="Music Mood Prediction",
    name="Histogram-based Gradient Boosting Regressor"
    )

In [24]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate

hgbt = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))
regression = hgbt.fit(X_train, y_train)

In [25]:
r2 = regression.score(X_test, y_test)
wandb.log({"Coefficient of Correlation (R^2)": r2})
# The outliers chart only supports univariate (i.e. 1D output) regression
# wandb.sklearn.plot_regressor(regression,
#                             X_train, X_test,
#                             y_train, y_test,
#                             model_name="hist_gradient_boosting_regressor")
wandb.sklearn.plot_learning_curve(regression, X_train, y_train)
wandb.sklearn.plot_summary_metrics(regression, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

param_grid = {
    "learning_rate": [0.1, 0.01, 0.001],
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "min_samples_split": [20, 35, 50],
    "min_samples_leaf": list(range(1,6))
}

cv_fold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(hgbt, param_grid, cv=cv_fold, n_jobs=-1, scoring="neg_mean_squared_error")
param_search_results = grid_search.fit(X_train, y_train)

best_model = param_search_results.best_estimator_
best_params = param_search_results.best_params_
wandb.log({"Coefficient of Correlation (R^2)": best_model.score(X_test, y_test)})

In [ ]:
wandb.sklearn.plot_learning_curve(best_model, X_train, y_train)
wandb.sklearn.plot_summary_metrics(best_model, X_train, y_train, X_test, y_test)

In [26]:
wandb.finish()

/home/tom/miniconda3/envs/cmpt353/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Coefficient of Correlation (R^2),▁
Coefficient of Correlation (R^2),0.02641
